In [18]:
#https://ru.wikipedia.org/wiki/%D0%9B%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F
import numpy as np
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

LEARNING_RATE = 0.01
EPOCH_GRADIENT = 225
EPOCH_NESTEROV = 25
EPOCH_RMSPROP = 25

data = load_iris()
df = data.data
df = df[data.target != 0]
#добавим на первое место столбец из единиц
df = np.insert(df, 0, 1, axis=1)
#вектор начальных параметров
t = np.array([1]*5, dtype=np.float64)
target = data.target[data.target != 0]

X_train, X_test, y_train, y_test = train_test_split(df, target)

def output(params, label=''):
    predictor = create_func(params)
    print('params of the model: ', params)
    y = y_test[:]
    y = y - 1
    predictions = []
    print(label + ':')
    print('predictions', 'y_test')
    for i in range(len(X_test)):
        print(predictor(X_test[i, :]), ' '*14, y[i])
        predictions.append(predictor(X_test[i, :]))
    print(label + ' accuracy_score = ', accuracy_score(y, np.array(predictions)))
    print(label + ' precision_score = ', precision_score(y, np.array(predictions)))
    print(label + ' recall_score = ', recall_score(y, np.array(predictions)))
    print('\n\n')

def sigmoid(t, x):
    z = sum(x * t)
    return 1 / (1 + np.exp(-z))

#созданинии функции-модели с полученными параметрами
def create_func(t):
    def f(x):
        return int(bool(sigmoid(t, x) > 0.5))
    return f

#градиент логарифма функции правдоподобия
def gradient(t):
    s = np.zeros(5)
    for i in range(len(X_train)):
        x = X_train[i,:]
        y = y_train[i] - 1
        k = y - sigmoid(t, x)
        s += k*x
    return s

def gradient_fit(t0):
    t = t0
    for i in range(EPOCH_GRADIENT):
        t += LEARNING_RATE * gradient(t)
    return t

def create_func(t):
    def f(x):
        return int(bool(sigmoid(t, x) > 0.5))
    return f

output(gradient_fit(t), label='gradient')


#Nesterov Accelerated Gradient
#v(t)=gamma*v(t-1)+eta*(1-gamma)*gradient(theta - gamma*v(t-1))
#theta=theta-v(t)

GAMMA = 0.9
v = np.zeros(5, dtype=np.float64)

def nesterov_fit(t0):
    global v
    t = t0
    for i in range(EPOCH_NESTEROV):
        v = GAMMA * v + LEARNING_RATE * (1 - GAMMA) * gradient(t - GAMMA * v)
        t += v
    return t

output(nesterov_fit(t), label='nesterov')

#RMSProp
#E(t) = gamma * E(t-1) + (1 - gamma) * gradient ^ 2
#teta = teta - eta/sqrt(E(t) + eps) * gradient

E = 0
GAMMA = 0.5
eps = 1e-5

def RMSProp_fit(t0):
    global E
    t = t0
    for i in range(EPOCH_RMSPROP):
        g = gradient(t)
        E = GAMMA * E + (1 - GAMMA) * np.linalg.norm(g) ** 2
        t += LEARNING_RATE/np.sqrt(E + eps) * g
    return t

output(RMSProp_fit(t), label='RMSProp')
    


params of the model:  [-3.28359746 -8.25880594 -5.36918474 10.6121432  10.34966641]
gradient:
predictions y_test
1                1
1                1
0                0
1                1
0                0
0                0
0                0
0                1
1                1
1                1
0                0
0                0
1                1
0                0
0                0
0                0
1                1
0                0
1                1
1                1
0                0
0                1
0                0
1                1
1                1
gradient accuracy_score =  0.92
gradient precision_score =  1.0
gradient recall_score =  0.8461538461538461



params of the model:  [-3.32990016 -8.22801144 -5.36765435 10.56649371 10.39207825]
nesterov:
predictions y_test
1                1
1                1
0                0
1                1
0                0
0                0
0                0
0                1
1                1
1                